<a href="https://colab.research.google.com/github/pranavsrinivas29/hyperparameter_opt/blob/main/LSTM_with_SGD_with_opt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install optuna

In [ ]:
import optuna
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Embedding, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.datasets import imdb

In [ ]:
# Load the IMDB dataset (as an example of text data)
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

# Preprocess the input data (sequence padding)
max_sequence_length = 200  # Adjust as needed
train_data = tf.keras.preprocessing.sequence.pad_sequences(train_data, maxlen=max_sequence_length)
test_data = tf.keras.preprocessing.sequence.pad_sequences(test_data, maxlen=max_sequence_length)

In [ ]:
# Define a function to create your LSTM-based model based on hyperparameters
def create_model(trial):
    model = Sequential()

    embedding_dim = trial.suggest_int('embedding_dim', 32, 256)
    lstm_units = trial.suggest_int('lstm_units', 32, 256)
    dense_units = trial.suggest_int('dense_units', 32, 256)
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.0, 0.5)

    model.add(Embedding(input_dim=10000, output_dim=embedding_dim, input_length=max_sequence_length))
    model.add(LSTM(units=lstm_units))
    model.add(Dense(units=dense_units, activation='relu'))
    model.add(tf.keras.layers.Dropout(rate=dropout_rate))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
# Define an objective function for Optuna
def objective(trial):
    model = create_model(trial)

    history = model.fit(train_data, train_labels, epochs=5, batch_size=64, validation_split=0.2, verbose=0)

    validation_loss = history.history['val_loss'][-1]

    return validation_loss

In [ ]:
# Create an Optuna study and optimize
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=3)  # You can adjust the number of trials


[I 2023-09-17 17:16:47,703] A new study created in memory with name: no-name-f260ae7d-7316-46d4-a5d2-366039389a4a
<ipython-input-5-df2e680410fb>:8: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate = trial.suggest_uniform('dropout_rate', 0.0, 0.5)
[I 2023-09-17 17:24:14,192] Trial 0 finished with value: 0.4444919526576996 and parameters: {'embedding_dim': 38, 'lstm_units': 128, 'dense_units': 173, 'dropout_rate': 0.04548013466988099}. Best is trial 0 with value: 0.4444919526576996.
[I 2023-09-17 17:26:22,955] Trial 1 finished with value: 0.4600178599357605 and parameters: {'embedding_dim': 93, 'lstm_units': 36, 'dense_units': 97, 'dropout_rate': 0.1021333207391858}. Best is trial 0 with value: 0.4444919526576996.
[I 2023-09-17 17:41:34,529] Trial 2 finished with value: 0.5167690515518188 and parameters: {'embedding_dim': 118, 'lstm_unit

In [ ]:
# Retrieve the best hyperparameters
best_hyperparameters = study.best_params
print("Best Hyperparameters:", best_hyperparameters)

Best Hyperparameters: {'embedding_dim': 38, 'lstm_units': 128, 'dense_units': 173, 'dropout_rate': 0.04548013466988099}
